<a href="https://colab.research.google.com/github/ryotaro1218/project1_2/blob/main/new_project2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import pandas as pd
import numpy as np


# Load data
pwt1001 = pd.read_stata('https://dataverse.nl/api/access/datafile/354098')

# Filter and select relevant columns
data = pwt1001.loc[pwt1001['country'].isin(["Canada",  "France", "Germany", "Italy", "Japan", "United Kingdom", "United States"])][['year', 'countrycode', 'rgdpna', 'rkna', 'emp', 'labsh', 'rtfpna', 'pop', 'avh', "country"]]
data = data.loc[(data['year'] >= 1995) & (data['year'] <= 2019)].dropna()

# Calculate additional columns
data['y_pc'] = np.log(data['rgdpna'] / data['emp'])  # GDP per worker
data['k_pc'] = np.log(data['rkna'] / data['emp'])  # Capital per worker
data['a'] = np.log(data['rtfpna'] )  # lnA
data['al'] = 1 - data['labsh']  # Capital share


# Order by year
data = data.sort_values('year')

# Group by isocode
grouped_data = data.groupby('countrycode')

# Calculate growth rates and Solow residual
data['gg'] = (grouped_data['y_pc'].diff() * 100)  # Growth rate of GDP per capita
data['gc'] = (grouped_data['k_pc'].diff() * 100)  # Growth rate of Capital per worker
data['ga'] = (grouped_data['a'].diff() * 100)  # Growth rate of A
data['algc'] = data['al'] * data['gc']
data['re'] = data['gg'] - data['ga'] -data['algc']
data['ts'] = data['ga'] / data['gg']
data['cs'] = data['algc'] / data['gg']

# Remove missing values
data = data.dropna()

# Calculate summary statistics
summary = data.groupby('country').agg({'gg': 'mean', 'cs': "mean", 'algc': "mean", 'ga': "mean", 'ts': "mean", 're': "mean"})

# Calculate additional summary statistics
summary['Growth Rate'] = summary['gg']
summary['TFP Growth'] = summary['ga']
summary['Residual error'] = summary['re']
summary['Capital Deepening'] = summary['algc']
summary['TFP Share'] = summary['ts']
summary['Capital Share'] = summary['cs']

summary = summary[['Growth Rate', 'TFP Growth', 'Capital Deepening', 'Residual error', 'TFP Share', 'Capital Share']]

summary.loc['Average'] = np.nan
summary.loc['Average', :] = summary.mean()

rounded_summary = summary.round(2)

# Print output
print(rounded_summary)

                Growth Rate  TFP Growth  Capital Deepening  Residual error  \
country                                                                      
Canada                 0.89        0.15               0.63            0.10   
France                 0.83        0.09               0.57            0.17   
Germany                0.66        0.36               0.44           -0.14   
Italy                 -0.04       -0.77               0.53            0.20   
Japan                  0.71        0.14               0.63           -0.06   
United Kingdom         1.02        0.23               0.57            0.21   
United States          1.54        0.68               0.78            0.08   
Average                0.80        0.13               0.59            0.08   

                TFP Share  Capital Share  
country                                   
Canada               2.13          -1.49  
France               0.78           0.19  
Germany              0.52           0.45  
Ital